In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Autism_spectrum_disorder_(ASD)/GSE148450'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Expression changes in immune and epigenetic gene pathways associated with nutritional metabolites in maternal blood from pregnancies resulting in autism and atypical neurodevelopment"
!Series_summary	"The prenatal period is a critical window to study factors involved in the development of autism spectrum disorder (ASD). Environmental factors, especially in utero nutrition, can interact with genetic risk for ASD, but how specific prenatal nutrients in mothers of children later diagnosed with ASD or non-typical development (Non-TD) associate with gestational gene expression events is poorly understood. Maternal blood collected prospectively during pregnancy provides a new opportunity to gain insights into nutritional effects on gene pathways and neurodevelopment. Using differential gene expression analysis, six transcripts associated with four genes (TGR-AS1, SQSTM1, HLA-C and RFESD) showed genome-wide significance (FDR q < 0.05) with child outcomes

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Checking gene expression data availability
is_gene_available = True

# Checking trait availability and setting keys
if 0 in [0, 1, 2, 3, 4]:
    if len(set(['diagnosis: ASD', 'diagnosis: NonTD', 'diagnosis: TD'])) > 1:
        trait_row = 0
if 1 in [0, 1, 2, 3, 4]:
    if len(set(['Sex: M', 'Sex: F'])) > 1:
        gender_row = 1

# No information on age row in the sample characteristics dictionary
# Thus, leaving age_row as None

# Defining data conversion functions

# Function to convert Autism_spectrum_disorder_(ASD) values
def convert_trait(value):
    if "ASD" in value:
        return 1
    elif "NonTD" in value or "TD" in value:
        return 0
    return None

# No data on age, hence convert_age will return None
def convert_age(value):
    return None

# Function to convert gender values
def convert_gender(value):
    if "M" in value:
        return 1
    elif "F" in value:
        return 0
    return None

# Saving cohort metadata
save_cohort_info('GSE148450', './preprocessed/Autism_spectrum_disorder_(ASD)/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical feature extraction step
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Autism_spectrum_disorder_(ASD)', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Autism_spectrum_disorder_(ASD)/trait_data/GSE148450.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM4471661': [1, 1], 'GSM4471662': [1, 1], 'GSM4471663': [1, 1], 'GSM4471664': [1, 1], 'GSM4471665': [1, 1], 'GSM4471666': [1, 1], 'GSM4471667': [1, 1], 'GSM4471668': [1, 1], 'GSM4471669': [1, 1], 'GSM4471670': [1, 0], 'GSM4471671': [1, 0], 'GSM4471672': [1, 1], 'GSM4471673': [1, 1], 'GSM4471674': [1, 1], 'GSM4471675': [1, 1], 'GSM4471676': [1, 1], 'GSM4471677': [1, 1], 'GSM4471678': [1, 1], 'GSM4471679': [1, 1], 'GSM4471680': [1, 1], 'GSM4471681': [1, 0], 'GSM4471682': [1, 1], 'GSM4471683': [1, 1], 'GSM4471684': [1, 1], 'GSM4471685': [1, 1], 'GSM4471686': [1, 0], 'GSM4471687': [1, 1], 'GSM4471688': [1, 1], 'GSM4471689': [1, 1], 'GSM4471690': [1, 1], 'GSM4471691': [1, 0], 'GSM4471692': [1, 0], 'GSM4471693': [1, 1], 'GSM4471694': [1, 0], 'GSM4471695': [1, 0], 'GSM4471696': [1, 1], 'GSM4471697': [1, 1], 'GSM4471698': [1, 1], 'GSM4471699': [1, 1], 'GSM4471700': [1, 0], 'GSM4471701': [1, 0], 'GSM4471702': [1, 0], 'GSM4471703': [1, 1], 'GSM4471704': [1, 1], 'GSM4471705': [1, 1], 'GSM44717

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])
    

gene_symbols = ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2ML1-AS1', 'A2ML1-AS2', 'A2MP1', 'A4GALT', 'A4GNT', 'AA06', 'AAAS', 'AACS', 'AACSP1', 'AADAC', 'AADACL2', 'AADACL2-AS1', 'AADACL3', 'AADACL4']


Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2ML1-AS1',
       'A2ML1-AS2', 'A2MP1', 'A4GALT', 'A4GNT', 'AA06', 'AAAS', 'AACS',
       'AACSP1', 'AADAC', 'AADACL2', 'AADACL2-AS1', 'AADACL3', 'AADACL4'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = False


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))
    

# This step is to be skipped as 'requires_gene_mapping' is False.


Gene annotation preview:
{'ID': ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1'], 'SPOT_ID': ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1']}


### Step 7: Data Normalization and Merging

In [7]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Autism_spectrum_disorder_(ASD)/gene_data/GSE148450.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Autism_spectrum_disorder_(ASD)')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE148450', './preprocessed/Autism_spectrum_disorder_(ASD)/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Autism_spectrum_disorder_(ASD)/GSE148450.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Autism_spectrum_disorder_(ASD)', the least common label is '1.0' with 56 occurrences. This represents 26.17% of the dataset.
The distribution of the feature 'Autism_spectrum_disorder_(ASD)' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 92 occurrences. This represents 42.99% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

